### Implement and test pytorch implementation of Multivariate Gaussian 

Generate some Gaussian 2D data and evaluate Gaussian in pytorch and scipy to test it. 

In [ ]:
import numpy as np 
import torch 
from torch.autograd import Variable
import scipy.stats

In [ ]:
# generate some 2D Gaussian samples 
d = 2
n = 10
mean = [1.] * d
cov = np.eye(d)
U = cov 

m = scipy.stats.multivariate_normal(mean=mean, cov=cov)

In [ ]:
# get some samples 
data = m.rvs(10)

In [ ]:
# now implement the pytorch version and compare the results 

# construct the pytorch Variables as batches 
X = Variable(torch.Tensor(data.tolist()))
mus = Variable(torch.ones(n, d))
Us = Variable(torch.zeros(n, d, d))
for idx in range(n): 
    Us[idx, ] = torch.eye(d)

In [ ]:
def multivariate_normal_pdf(X, mus, Us, log=False):
    # dimension of the Gaussian 
    D = mus.size()[1]
    N = mus.size()[0]
    
    # get the precision matrices over batches using matrix multiplication: S^-1 = U'U
    Sin = torch.bmm(torch.transpose(Us, 1, 2), Us)
    
    norm_const = Variable(torch.zeros(N, 1))
    log_probs = Variable(torch.zeros(N, 1))
    
    for idx in range(N): 
        norm_const[idx] = (torch.log(torch.diag(Us[idx, ])).sum(-1) + (D / 2) * np.log(2 * np.pi)).unsqueeze(-1)
        diff = (X[idx, ] - mus[idx, ]).unsqueeze(-1)
        log_probs[idx] = 0.5 * torch.mm(torch.transpose(diff, 0, 1), torch.mm(Sin[idx, ], diff))
        
    ps = -(norm_const + log_probs)
    log_probs = ps 
    
    if log:
        return log_probs
    else: 
        return torch.exp(log_probs)

pdata_torch = multivariate_normal_pdf(X, mus, Us, log=True)

In [ ]:
# evaluate the scipy pdf as ground truth 
pdata_scipy = np.log(m.pdf(data))

In [ ]:
pdata_torch

In [ ]:
pdata_scipy